# Extracción de datos estruturados de SECOP I

In [3]:
# Instalación de la libreria sodapy
#!pip install sodapy

In [1]:
# Importar librerias
import os
from sodapy import Socrata
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
# Configurar la conexión con la base de datos de Procesos de SECOP I
# https://www.datos.gov.co/resource/xvdy-vvsk.json
socrata_domain = 'www.datos.gov.co'
socrata_dataset_identifier = 'xvdy-vvsk'
socrata_token = os.environ.get("SzZ4759qpp2oHfKaHnlgR5T1n")
client = Socrata(socrata_domain, socrata_token)

In [4]:
# Extraer la metadata de la base de datos
# Esto permite obtener información general que ayuda en el diseño de las consultas
metadata = client.get_metadata(socrata_dataset_identifier)
col_names = [x['fieldName'] for x in metadata['columns']]
col_names

['uid',
 'anno_cargue_secop',
 'anno_firma_del_contrato',
 'nivel_entidad',
 'orden_entidad',
 'nombre_de_la_entidad',
 'nit_de_la_entidad',
 'c_digo_de_la_entidad',
 'id_tipo_de_proceso',
 'tipo_de_proceso',
 'estado_del_proceso',
 'causal_de_otras_formas_de',
 'id_regimen_de_contratacion',
 'regimen_de_contratacion',
 'id_objeto_a_contratar',
 'objeto_a_contratar',
 'detalle_del_objeto_a_contratar',
 'tipo_de_contrato',
 'municipio_obtencion',
 'municipio_entrega',
 'municipios_ejecucion',
 'fecha_de_cargue_en_el_secop',
 'numero_de_constancia',
 'numero_de_proceso',
 'numero_del_contrato',
 'cuantia_proceso',
 'id_grupo',
 'nombre_grupo',
 'id_familia',
 'nombre_familia',
 'id_clase',
 'nombre_clase',
 'id_ajudicacion',
 'tipo_identifi_del_contratista',
 'identificacion_del_contratista',
 'nom_raz_social_contratista',
 'dpto_y_muni_contratista',
 'tipo_doc_representante_legal',
 'identific_del_represen_legal',
 'nombre_del_represen_legal',
 'fecha_de_firma_del_contrato',
 'fecha_ini

In [59]:
# Diseño de la consulta en SQL
query = """
SELECT 
    uid,
    nombre_de_la_entidad,
    departamento_entidad,
    orden_entidad,
    tipo_de_proceso,
    objeto_a_contratar,
    UPPER(detalle_del_objeto_a_contratar) AS detalle_objeto,
    cuantia_proceso,
    cuantia_contrato,
    valor_total_de_adiciones,
    valor_contrato_con_adiciones,
    anno_firma_del_contrato AS anno_firma,
    fecha_de_firma_del_contrato AS fecha_fima,
    fecha_ini_ejec_contrato,
    plazo_de_ejec_del_contrato,
    rango_de_ejec_del_contrato,
    tiempo_adiciones_en_dias,
    tiempo_adiciones_en_meses,
    fecha_fin_ejec_contrato
WHERE
    anno_firma IS NOT NULL
    AND fecha_fima IS NOT NULL
    AND detalle_objeto IS NOT NULL

    AND id_familia = '9511'
    AND estado_del_proceso = 'Liquidado'
    AND anno_firma NOT IN ('2010','2011','2012','2013','2021','2022')
    AND cuantia_proceso > 20000000
    AND cuantia_contrato > 20000000
    AND regimen_de_contratacion != 'Régimen Especial'
    AND tipo_de_proceso IN ('Licitación obra pública','Licitación Pública')

    AND detalle_objeto NOT LIKE '%MEZCLA ASFÁLTICA%'
    AND detalle_objeto NOT LIKE '%REDUCTORES DE VELOCIDAD%'
    AND detalle_objeto NOT LIKE '%MUROS DE CONTENCIÓN%'
    AND detalle_objeto NOT LIKE '%DEMOLICIÓN%'
    AND detalle_objeto NOT LIKE '%RESTAURACIÓN ESTACIÓN FÉRREA%'
    AND detalle_objeto NOT LIKE '%CONSTRUCCIÓN DE REDUCTORES DE VELOCIDAD%'
    AND detalle_objeto NOT LIKE '%REDUCTORES DE VELOCIDAD%'
    AND detalle_objeto NOT LIKE '%CONSTRUCCIÓN DE REDUCTORES DE VELOCIDAD%'
    AND detalle_objeto NOT LIKE '%BARANDAS%'
    AND detalle_objeto NOT LIKE '%MUROS DE CONTENCIÓN%'
    AND detalle_objeto NOT LIKE '%DEMOLICIÓN%'
    AND detalle_objeto NOT LIKE '%RESTAURACIÓN ESTACIÓN FÉRREA%'
    AND detalle_objeto NOT LIKE '%SEÑALIZACIÓN%'
    AND detalle_objeto NOT LIKE '%REHABILITACIÓN Y CONSERVACIÓN PUENTE%'
    AND detalle_objeto NOT LIKE '%MANTENIMIENTO TÚNEL%'
    AND detalle_objeto NOT LIKE '%ALCANTARILLA%'
    AND detalle_objeto NOT LIKE '%MANO DE OBRA%'
    AND detalle_objeto NOT LIKE '%BOX CULVERT%'
    AND detalle_objeto NOT LIKE '%PUENTES COLGANTES%'
    AND detalle_objeto NOT LIKE '%CASCO URBANO%'
    AND detalle_objeto NOT LIKE '%VÍAS URBANAS%'
    AND detalle_objeto NOT LIKE '%EJERCITO%'
    AND detalle_objeto NOT LIKE '%SEMAFORIZACIÓN%'
    AND detalle_objeto NOT LIKE '%DEMOLICIONES%'
    AND detalle_objeto NOT LIKE '%MURO%'
    AND detalle_objeto NOT LIKE '%CICLOVÍAS%'
    AND detalle_objeto NOT LIKE '%CICLORUTA%'
    AND detalle_objeto NOT LIKE '%RESIDUOS SÓLIDOS%'
    AND detalle_objeto NOT LIKE '%DESMONTE Y LIMPIEZA%'
    AND detalle_objeto NOT LIKE '%AULAS%'
    
LIMIT
    1000
"""
results = client.get(socrata_dataset_identifier, query=query)
results = pd.DataFrame.from_dict(results)
results.to_csv('data/contratos_familia_9511.csv', index=False)
df = results.copy()
print("El numero de contratos consultados: {}".format(results.shape[0]))
results.head()

El numero de contratos consultados: 720


,uid,nombre_de_la_entidad,departamento_entidad,orden_entidad,tipo_de_proceso,objeto_a_contratar,detalle_objeto,cuantia_proceso,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,anno_firma,fecha_fima,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato
0,15-1-1488216430156,VALLE DEL CAUCA ALCALDÍA MUNICIPIO DE LA UNIÓN,Valle del Cauca,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",CONSTRUCCION DE PAVIMENTO RIGIDO EN LA CL 13 C...,285341776,285151179,0,285151179,2015,2015-11-18,2015-12-14,60,D,30,0,2016-03-13 00:00:00
1,15-1-1445654095917,ANTIOQUIA ALCALDÍA MUNICIPIO DE MEDELLÍN,Antioquia,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",CONFORMACIÓN PARCHEO PAVIMENTACIÓN REPAVIMENTA...,1779588615,1750717720,0,1750717720,2015,2015-10-09,2015-11-03,3,M,0,0,2016-02-03 00:00:00
2,18-21-74068240139,CASANARE ALCALDÍA MUNICIPIO DE SABANALARGA,Casanare,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación obra pública,"Terrenos, Edificios, Estructuras y Vías",REALIZAR LA CONSTRUCCIÓN DE OBRAS DE DRENAJE Y...,279116919,279010379,44271346,323281725,2019,2019-02-01,2019-02-01,4,M,15,0,2019-06-01 00:00:00
3,17-1-1792116571655,CAQUETÁ ALCALDÍA MUNICIPIO DE CARTAGENA DEL C...,Caquetá,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",EJECUCION DEL PROGRAMA DE RED DE VIAS TERCIARI...,878217589,878217589,0,878217589,2017,2017-10-18,2017-10-20,2,M,248,0,2018-08-20 00:00:00
4,15-1-1398693847921,BOYACÁ ALCALDÍA MUNICIPIO DE CHIQUINQUIRÁ,Boyacá,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",MANTENIMIENTO Y REPARCHEO DE VÍAS DEL PERÍMETR...,360000000,359948173,179897140,539845313,2015,2015-07-22,2015-07-24,90,D,0,0,2015-10-22 00:00:00


In [104]:
#df.to_csv('data/contratos_familia_9511.csv', index=False)
df = pd.read_csv('data/contratos_familia_9511.csv')
df.head()

,uid,nombre_de_la_entidad,departamento_entidad,orden_entidad,tipo_de_proceso,objeto_a_contratar,detalle_objeto,cuantia_proceso,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,anno_firma,fecha_fima,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato
0,15-1-1488216430156,VALLE DEL CAUCA ALCALDÍA MUNICIPIO DE LA UNIÓN,Valle del Cauca,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",CONSTRUCCION DE PAVIMENTO RIGIDO EN LA CL 13 C...,2.853418e+08,2.851512e+08,0.0,2.851512e+08,2015,2015-11-18,2015-12-14,60,D,30,0,2016-03-13 00:00:00
1,15-1-1445654095917,ANTIOQUIA ALCALDÍA MUNICIPIO DE MEDELLÍN,Antioquia,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",CONFORMACIÓN PARCHEO PAVIMENTACIÓN REPAVIMENTA...,1.779589e+09,1.750718e+09,0.0,1.750718e+09,2015,2015-10-09,2015-11-03,3,M,0,0,2016-02-03 00:00:00
2,18-21-74068240139,CASANARE ALCALDÍA MUNICIPIO DE SABANALARGA,Casanare,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación obra pública,"Terrenos, Edificios, Estructuras y Vías",REALIZAR LA CONSTRUCCIÓN DE OBRAS DE DRENAJE Y...,2.791169e+08,2.790104e+08,44271346.0,3.232817e+08,2019,2019-02-01,2019-02-01,4,M,15,0,2019-06-01 00:00:00
3,17-1-1792116571655,CAQUETÁ ALCALDÍA MUNICIPIO DE CARTAGENA DEL C...,Caquetá,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",EJECUCION DEL PROGRAMA DE RED DE VIAS TERCIARI...,8.782176e+08,8.782176e+08,0.0,8.782176e+08,2017,2017-10-18,2017-10-20,2,M,248,0,2018-08-20 00:00:00
4,15-1-1398693847921,BOYACÁ ALCALDÍA MUNICIPIO DE CHIQUINQUIRÁ,Boyacá,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,Licitación Pública,"Terrenos, Edificios, Estructuras y Vías",MANTENIMIENTO Y REPARCHEO DE VÍAS DEL PERÍMETR...,3.600000e+08,3.599482e+08,179897140.0,5.398453e+08,2015,2015-07-22,2015-07-24,90,D,0,0,2015-10-22 00:00:00


In [105]:
def process_data(df):
    """
    Aplica todas las tareas de procesamiento y estandarización
    """
    
    # Ajustar el tipo de columna 
    df = (df
     .astype({'cuantia_proceso':'float',
              'cuantia_contrato':'float',
              'valor_total_de_adiciones':'float',
              'valor_contrato_con_adiciones':'float',
              'anno_firma':'int32',
              'plazo_de_ejec_del_contrato':'int32',
              'tiempo_adiciones_en_dias':'int32',
              'tiempo_adiciones_en_meses':'int32',
              'fecha_fima':'datetime64[ns]',
              'fecha_ini_ejec_contrato':'datetime64[ns]',
              'fecha_fin_ejec_contrato':'datetime64[ns]'}))
    
    df.rename(columns={'cuantia_proceso':'ESTIMATED_COST',
                       'plazo_de_ejec_del_contrato':'ORIGINAL_DEADLINE',
                       'cuantia_contrato':'CONTRACT_VALUE',
                       'valor_contrato_con_adiciones':'FINAL_COST',
                       'valor_total_de_adiciones':'ADDITIONAL_COST',
                       'anno_firma':'YEAR',
                       'orden_entidad': 'MUNICIPALITY_TYPE',
                       'departamento_entidad': 'DEPARTMENT',
                       'fecha_fima':'CONTRACT_DATE',
                       'fecha_ini_ejec_contrato':'START_DATE',
                       'fecha_fin_ejec_contrato':'END_DATE',
                       'nombre_de_la_entidad':'ENTITY_NAME',
                       'municipio_entidad':'MUNICIPALITY',
                       'tipo_de_proceso':'PROCESS_TYPE',
                       'objeto_a_contratar':'CONTRACT_OBJECT',
                       'detalle_objeto':'OBJETC_DETAIL',
                       'uid':'CONTRACT_ID'}, 
              inplace=True)
    
    # Escalar el valor de los contratos a SMMLV
    cuantia_col = ['ESTIMATED_COST', 'CONTRACT_VALUE', 'ADDITIONAL_COST', 'FINAL_COST']
    
    salario_minimo = {2014:616000, 
                      2015:644350,
                      2016:689455,
                      2017:737717,
                      2018:781242,
                      2019:828116,
                      2020:877803}
    
    for col in cuantia_col:
        temp = [val/salario_minimo[anno] for val,anno in zip(df[col],df['YEAR'])]
        df[col] = temp
        del temp
    
    # Convertir el plazo del contrato en dias 
    df.loc[df['rango_de_ejec_del_contrato'] == 'M', 'ORIGINAL_DEADLINE'] = df['ORIGINAL_DEADLINE']*30
    df.drop(columns=['rango_de_ejec_del_contrato'], inplace=True)

    # Unificar las adiciones en tiempo
    df['ADDITIONAL_TIME'] = df['tiempo_adiciones_en_dias'] + df['tiempo_adiciones_en_meses']*30
    df.drop(columns=['tiempo_adiciones_en_dias', 'tiempo_adiciones_en_meses'], inplace=True)

    # Calcular la duración total del proyecto
    df['FINAL_DEADLINE'] = df['ORIGINAL_DEADLINE'] + df['ADDITIONAL_TIME']
    
    # Calcular la intensidad del proyecto
    df['PROJECT_INTENSITY'] = df['CONTRACT_VALUE']/df['ORIGINAL_DEADLINE']

    # Calcular el award growth
    df['AWARD_GROWTH'] = ((df['CONTRACT_VALUE'] - df['ESTIMATED_COST'])/df['ESTIMATED_COST'])*100

    # Calcular la desviación en costo
    df['COST_DEVIATION'] = (df['FINAL_COST'] - df['CONTRACT_VALUE'])/df['CONTRACT_VALUE']

    # Calcular la desviación en tiempo
    df['TIME_DEVIATION'] = (df['FINAL_DEADLINE'] - df['ORIGINAL_DEADLINE'])/df['ORIGINAL_DEADLINE']

    df['OWNER'] = df['MUNICIPALITY_TYPE']
    # Estandarizar el tipo de municipio
    original_municipality_type = ['TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1',
                                  'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2',
                                  'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 3',
                                  'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 4',
                                  'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 5',
                                  'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6',
                                  'TERRITORIAL DEPARTAMENTAL CENTRALIZADO',
                                  'TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO',
                                  'DISTRITO CAPITAL',
                                  'NACIONAL CENTRALIZADO']

    new_municipality_type = ['TYPE_1', 'TYPE_2', 'TYPE_3', 'TYPE_4', 'TYPE_5', 'TYPE_6', 'OTHER', 'OTHER', 'OTHER', 'OTHER']
    df['MUNICIPALITY_TYPE'] = df['MUNICIPALITY_TYPE'].replace(original_municipality_type, new_municipality_type)

    # Definir el tipo de entidad departamental, nacional o territorial
    new_owner = ['MUNICIPALITY', 'MUNICIPALITY', 'MUNICIPALITY', 'MUNICIPALITY', 'MUNICIPALITY', 'MUNICIPALITY', 'DEPARTMENT_GOVERNMENT', 'DEPARTMENT_GOVERNMENT', 'OTHER', 'OTHER']
    df['OWNER'] =  df['OWNER'].replace(original_municipality_type, new_owner)

    # Regiones 
    AMAZONIA = ['Amazonas', 'Caquetá', 'Putumayo', 'Guainía', 'Guaviare', 'Vaupés']
    ORINOQUIA = ['Meta', 'Arauca', 'Casanare', 'Vichada']
    ANDINA = ['Antioquia', 'Boyacá', 'Caldas', 'Cundinamarca', 'Huila', 'Norte De Santander', 'Quindío', 'Risaralda', 'Santander', 'Tolima', 'Bogotá D.C.']
    CARIBE = ['Atlántico', 'Bolívar', 'Cesar', 'Córdoba', 'La Guajira', 'Magdalena', 'Sucre', 'San Andrés, Providencia y Santa Catalina']
    PACIFICA = ['Cauca', 'Valle del Cauca', 'Chocó', 'Nariño']

    # Crear un variable que asigne la región en función del departamento
    df['REGION'] = df['DEPARTMENT'].apply(lambda x: 'AMAZONIA' if x in AMAZONIA else
                                            'ORINOQUIA' if x in ORINOQUIA else
                                            'ANDINA' if x in ANDINA else
                                            'CARIBE' if x in CARIBE else
                                            'PACIFICA' if x in PACIFICA else
                                            'OTRA')

    # Crear una variable de tipo de pro

    # lower to uppercase
    df['DEPARTMENT'] = df['DEPARTMENT'].str.upper()
    df['PROCESS_TYPE'] = df['PROCESS_TYPE'].str.upper()
    df['CONTRACT_OBJECT'] = df['CONTRACT_OBJECT'].str.upper()   
    df['OBJETC_DETAIL'] = df['OBJETC_DETAIL'].str.upper()

    return df

In [106]:
process_data(df)

,CONTRACT_ID,ENTITY_NAME,DEPARTMENT,MUNICIPALITY_TYPE,PROCESS_TYPE,CONTRACT_OBJECT,OBJETC_DETAIL,ESTIMATED_COST,CONTRACT_VALUE,ADDITIONAL_COST,FINAL_COST,YEAR,CONTRACT_DATE,START_DATE,ORIGINAL_DEADLINE,END_DATE,ADDITIONAL_TIME,FINAL_DEADLINE,PROJECT_INTENSITY,AWARD_GROWTH,COST_DEVIATION,TIME_DEVIATION,OWNER,REGION
0,15-1-1488216430156,VALLE DEL CAUCA ALCALDÍA MUNICIPIO DE LA UNIÓN,VALLE DEL CAUCA,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONSTRUCCION DE PAVIMENTO RIGIDO EN LA CL 13 C...,442.836620,442.540823,0.000000,442.540823,2015,2015-11-18,2015-12-14,60,2016-03-13,30,90,7.375680,-0.066796,0.000000,0.500000,MUNICIPALITY,PACIFICA
1,15-1-1445654095917,ANTIOQUIA ALCALDÍA MUNICIPIO DE MEDELLÍN,ANTIOQUIA,TYPE_2,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONFORMACIÓN PARCHEO PAVIMENTACIÓN REPAVIMENTA...,2761.835361,2717.029130,0.000000,2717.029130,2015,2015-10-09,2015-11-03,90,2016-02-03,0,90,30.189213,-1.622335,0.000000,0.000000,MUNICIPALITY,ANDINA
2,18-21-74068240139,CASANARE ALCALDÍA MUNICIPIO DE SABANALARGA,CASANARE,TYPE_6,LICITACIÓN OBRA PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",REALIZAR LA CONSTRUCCIÓN DE OBRAS DE DRENAJE Y...,337.050509,336.921855,53.460320,390.382175,2019,2019-02-01,2019-02-01,120,2019-06-01,15,135,2.807682,-0.038170,0.158673,0.125000,MUNICIPALITY,ORINOQUIA
3,17-1-1792116571655,CAQUETÁ ALCALDÍA MUNICIPIO DE CARTAGENA DEL C...,CAQUETÁ,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",EJECUCION DEL PROGRAMA DE RED DE VIAS TERCIARI...,1190.453235,1190.453235,0.000000,1190.453235,2017,2017-10-18,2017-10-20,60,2018-08-20,248,308,19.840887,0.000000,0.000000,4.133333,MUNICIPALITY,AMAZONIA
4,15-1-1398693847921,BOYACÁ ALCALDÍA MUNICIPIO DE CHIQUINQUIRÁ,BOYACÁ,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",MANTENIMIENTO Y REPARCHEO DE VÍAS DEL PERÍMETR...,558.702568,558.622135,279.191651,837.813786,2015,2015-07-22,2015-07-24,90,2015-10-22,0,90,6.206913,-0.014396,0.499786,0.000000,MUNICIPALITY,ANDINA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,17-1-1832277150408,SANTANDER ALCALDÍA MUNICIPIO DE EL CARMEN DE ...,SANTANDER,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",MEJORAMIENTO DE VÍAS TERCIARIAS MEDIANTE EL US...,1934.292468,1930.894796,0.000000,1930.894796,2018,2018-01-17,2018-03-05,150,2018-08-05,0,150,12.872632,-0.175655,0.000000,0.000000,MUNICIPALITY,ANDINA
716,15-1-1445384111788,TOLIMA GOBERNACIÓN,TOLIMA,OTHER,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONTRATAR LA PAVIMENTACIÓN EN CONCRETO RÍGIDO ...,2718.118965,2644.822707,0.000000,2644.822707,2015,2015-12-09,2015-12-09,90,2016-04-22,45,135,29.386919,-2.696580,0.000000,0.500000,DEPARTMENT_GOVERNMENT,ANDINA
717,18-21-63338295928,ANTIOQUIA ALCALDÍA MUNICIPIO DE DON MATÍAS,ANTIOQUIA,TYPE_6,LICITACIÓN OBRA PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",TERMINACIÓN DEL PUENTE VEHÍCULAR DE CONEXIÓN V...,1545.534761,1545.534761,634.419826,2179.954588,2018,2018-12-20,2019-02-19,120,2019-10-19,120,240,12.879456,0.000000,0.410486,1.000000,MUNICIPALITY,ANDINA
718,16-1-1618435203961,INSTITUTO NACIONAL DE VÍAS INVIAS,BOGOTÁ D.C.,OTHER,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",MANTENIMIENTO RUTINARIO VIAS A CARGO DEL INSTI...,19053.356627,686.403081,304.955017,991.358097,2016,2016-10-14,2016-11-05,600,2019-04-05,295,895,1.144005,-96.397469,0.444280,0.491667,OTHER,ANDINA


https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-I/xvdy-vvsk

https://dev.socrata.com/foundry/www.datos.gov.co/xvdy-vvsk

https://dev.socrata.com/docs/queries/

https://www.w3schools.com/sql/sql_examples.asp
